In [1]:
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
# 二维互相关运算
def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i: i + h, j: j + w] * K).sum()
    return Y

In [3]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])

In [4]:
X, K

(tensor([[0., 1., 2.],
         [3., 4., 5.],
         [6., 7., 8.]]),
 tensor([[0., 1.],
         [2., 3.]]))

In [5]:
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [7]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
        
    def forward(self, X):
        return corr2d(X, self.weight) + self.bias

In [14]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [25]:
K = torch.tensor([[1.0, -1.0]])
K

tensor([[ 1., -1.]])

In [26]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [27]:
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
X, Y

(tensor([[[[1., 1., 0., 0., 0., 0., 1., 1.],
           [1., 1., 0., 0., 0., 0., 1., 1.],
           [1., 1., 0., 0., 0., 0., 1., 1.],
           [1., 1., 0., 0., 0., 0., 1., 1.],
           [1., 1., 0., 0., 0., 0., 1., 1.],
           [1., 1., 0., 0., 0., 0., 1., 1.]]]]),
 tensor([[[[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
           [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
           [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
           [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
           [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
           [ 0.,  1.,  0.,  0.,  0., -1.,  0.]]]]))

In [30]:
for i in range(20):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad
    print(f'batch {i + 1}, loss {l.sum(): .2f}')

batch 1, loss  0.18
batch 2, loss  0.11
batch 3, loss  0.07
batch 4, loss  0.05
batch 5, loss  0.03
batch 6, loss  0.02
batch 7, loss  0.01
batch 8, loss  0.01
batch 9, loss  0.00
batch 10, loss  0.00
batch 11, loss  0.00
batch 12, loss  0.00
batch 13, loss  0.00
batch 14, loss  0.00
batch 15, loss  0.00
batch 16, loss  0.00
batch 17, loss  0.00
batch 18, loss  0.00
batch 19, loss  0.00
batch 20, loss  0.00


In [33]:
conv2d.weight

Parameter containing:
tensor([[[[ 1.0006, -0.9994]]]], requires_grad=True)

In [56]:
# 填充
conv2d_1 = nn.Conv2d(1, 1, kernel_size=3, padding=1)
conv2d_2 = nn.Conv2d(1, 1, kernel_size=3, padding=0)

X = torch.rand((8, 8)).reshape(1, 1, 8, 8)
Y1 = conv2d_1(X)
Y2 = conv2d_2(X)
Y1.shape, Y2.shape

(torch.Size([1, 1, 8, 8]), torch.Size([1, 1, 6, 6]))

In [65]:
# 步幅
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
Y = conv2d(X)
print(Y.shape)

conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=4)
Y = conv2d(X)
print(Y.shape)

conv2d = nn.Conv2d(1, 1, kernel_size=(1, 3), padding=(0, 1), stride=(2, 3))
Y = conv2d(X)
print(Y.shape)

torch.Size([1, 1, 4, 4])
torch.Size([1, 1, 2, 2])
torch.Size([1, 1, 4, 3])
